In [ ]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp


| Fabryka    | Partia alu | Partia drewniana | Max czas tygodniowo |
|------------|------------|------------------|---------------------|
| F1         | 1 h.       | X                | 4 h.                |
| F2         | X          | 2 h.             | 12 h.               |
| F3         | 3 h.       | 2 h.             | 18 h.               |
|            |            |                  |                     |
|**Zysk**    | 3000\$     | 5000\$           |                     |

In [26]:
from pulp import *
#print(pulp.__version__)
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, FloatSlider, IntSlider
import ipywidgets as widgets
import pandas as pd

style = {'description_width': '200px'}
layout= Layout(width='50%', height='auto',grid_gap="50px")


zysk_alu_slider = FloatSlider(min=500,max=10000,value=3000,description="Zysk alu",style=style,layout=layout,step=100)
zysk_drewniane_slider = FloatSlider(min=500,max=10000,value=5000,description="Zysk drewniane",style=style,layout=layout,step=100)
godz_alu_F1_slider = FloatSlider(min=0,max=10,value=1,description="Godziny alu F1",style=style,layout=layout,step=0.5)
godz_alu_F2_slider = FloatSlider(min=0,max=10,value=0,description="Godziny alu F2",style=style,layout=layout,step=0.5)
godz_alu_F3_slider = FloatSlider(min=0,max=10,value=3,description="Godziny alu F3",style=style,layout=layout,step=0.5)
godz_drewniane_F1_slider = FloatSlider(min=0,max=10,value=0,description="Godziny drewniane F1",style=style,layout=layout,step=0.5)
godz_drewniane_F2_slider = FloatSlider(min=0,max=10,value=2,description="Godziny drewniane F2",style=style,layout=layout,step=0.5)
godz_drewniane_F3_slider = FloatSlider(min=0,max=10,value=2,description="Godziny drewniane F3",style=style,layout=layout,step=0.5)
maks_godz_F1_slider = FloatSlider(min=0,max=50,value=4,description="Maks godz. F1",style=style,layout=layout,step=0.5)
maks_godz_F2_slider = FloatSlider(min=0,max=50,value=12,description="Maks godz. F2",style=style,layout=layout,step=0.5)
maks_godz_F3_slider = FloatSlider(min=0,max=50,value=18,description="Maks godz. F3",style=style,layout=layout,step=0.5)

def produkcja_okien(
    zysk_alu = 3000,
    zysk_drewniane = 5000,
    godz_alu_F1 = 1,
    godz_alu_F2 = 0,
    godz_alu_F3 = 3,
    godz_drewniane_F1 = 0,
    godz_drewniane_F2 = 2,
    godz_drewniane_F3 = 2,
    maks_godz_F1 = 4,
    maks_godz_F2 = 12,
    maks_godz_F3 = 18
    ):
  
    
    prob = LpProblem("Maksymalizacja_zysku_z_okien",LpMaximize)

    x1=LpVariable("partie_alu",0,None,LpInteger)
    x2=LpVariable("partie_drewniane",0,None,LpInteger)

    prob += zysk_alu*x1 + zysk_drewniane*x2   #zysk

    prob += godz_alu_F1*x1 + godz_drewniane_F1*x2 <= maks_godz_F1, "Maks godziny/tyg w F1"
    prob += godz_alu_F2*x1 + godz_drewniane_F2*x2 <= maks_godz_F2, "Maks godziny/tyg w F2"
    prob += godz_alu_F3*x1 + godz_drewniane_F3*x2 <= maks_godz_F3, "Maks godziny/tyg w F3"

    #gdy 0 godz. => brak produkcji
    if godz_alu_F1 == 0 and godz_alu_F2 == 0 and godz_alu_F3 == 0: prob += x1 == 0
    if godz_drewniane_F1 == 0 and godz_drewniane_F2 == 0 and godz_drewniane_F3 == 0: prob += x2 == 0

    prob.writeLP("okna.lp")
    prob.solve()

    varsdict = {}

    for v in prob.variables():
        varsdict[v.name] = str(v.varValue)

    #!cat "okna.lp"

    print()
    print("Status:", LpStatus[prob.status])
    print("Końcowy zysk {}$, okna: {}".format(value(prob.objective),varsdict))
    print()
    shadows = [{'name':name, 'shadow price': c.pi, "slack": c.slack} for name, c in prob.constraints.items()]
    print("*"*16+"Shadow Prices"+"*"*16)
    print(pd.DataFrame(shadows))
    print("*"*45)


interact(
    produkcja_okien,zysk_alu = zysk_alu_slider,
    zysk_drewniane = zysk_drewniane_slider,
    godz_alu_F1 = godz_alu_F1_slider,
    godz_alu_F2 = godz_alu_F2_slider,
    godz_alu_F3 = godz_alu_F3_slider,
    godz_drewniane_F1 = godz_drewniane_F1_slider,
    godz_drewniane_F2 = godz_drewniane_F2_slider,
    godz_drewniane_F3 = godz_drewniane_F3_slider,
    maks_godz_F1 = maks_godz_F1_slider,
    maks_godz_F2 = maks_godz_F2_slider,
    maks_godz_F3 = maks_godz_F3_slider
)

interactive(children=(FloatSlider(value=3000.0, description='Zysk alu', layout=Layout(grid_gap='50px', height=…

<function __main__.produkcja_okien>